In [1]:
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
%matplotlib inline

In [2]:
import numpy as np

In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 10

In [4]:
num_topics = 10
num_kelly_instruments=44

In [5]:
cv_thresh = '2007-02-01'

In [6]:
test_thresh = '2010-02-01'

In [7]:
num_inputs = (num_kelly_instruments + num_topics)

In [8]:
num_factors = 4

In [9]:
if False:
    comb_df = pd.read_csv("s3://senior-essay-economics/transformed-data/factor-model/combined.csv", index_col=[0,1])
    comb_df = comb_df.dropna(how='any', axis=0)
    comb_df.to_csv("combined-local.csv")
else:
    comb_df = pd.read_csv("combined-local.csv", index_col=[0,1])
train_df = comb_df.loc[:cv_thresh]
cv_df = comb_df.loc[cv_thresh:test_thresh]
test_df = comb_df.loc[test_thresh:]

In [10]:
def dataset_generator(df, batched=True):
    if batched:
        for dat in set(df.index.get_level_values(0)):
            mtx = df.loc[dat]
            ret = tf.cast(mtx[['RET']], tf.float32)
            rest = mtx.drop("RET", axis=1)
            inp_ivs = tf.cast(rest, tf.float32)
            yield rest.values, mtx[['RET']].values
    else:
        pass
        #pnl = df.to_panel()
        #df[]
        #yield (df.drop("RET", axis=1).values, df[['RET']].values)

In [11]:
#pnl = train_df.to_panel()

In [12]:
TRAINING=False

In [13]:
if TRAINING:
    the_dataset=train_df
else:
    the_dataset = test_df

In [14]:
display_step = len(set(the_dataset.index.get_level_values(0))) - 1

In [15]:
ds = tf.data.Dataset.from_generator(lambda: dataset_generator(the_dataset),
                                    output_types=(tf.float32, tf.float32), output_shapes=([None, num_topics+num_kelly_instruments], [None, 1]))


In [16]:
#ds = ds.shuffle(1000)

In [17]:
if TRAINING:
    ds = ds.repeat(training_epochs)

In [18]:
iterator = ds.make_one_shot_iterator()

In [19]:
next_element = iterator.get_next()

In [20]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [21]:
X,R = next_element

In [22]:
#X = tf.placeholder(tf.float32, [None, num_inputs])

In [23]:
#R = tf.placeholder(tf.float32, [None, 1])

In [24]:
with tf.variable_scope("to_factors", reuse=tf.AUTO_REUSE, initializer=tf.contrib.layers.xavier_initializer()):
    W0 = tf.get_variable("W", shape = [num_inputs, num_factors],
                        regularizer=None, trainable=True)
    F = tf.matmul(X, W0)
    F_ext = tf.pad(F, [[0,0], [0,1]], constant_values=1)
    Frethat = tf.matrix_solve_ls(F_ext, R, fast=True)
    Rpred = tf.matmul(F_ext,Frethat)

In [25]:
saver = tf.train.Saver()

In [26]:
with tf.name_scope("loss"):
    eps = (R - Rpred)
    loss = tf.reduce_sum(eps * eps)
    # regularizer = tf.contrib.layers.l2_regularizer(lambda_l2)
    #penalty_new = tf.contrib.layers.apply_regularization(regularizer, new_vars)
    #loss = loss + ...

In [27]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

In [28]:
train_op = optimizer.minimize(loss)

In [29]:
mean_ret = tf.reduce_mean(R)

In [30]:
per_var = tf.reduce_sum((R - mean_ret)*(R-mean_ret))

In [31]:
#accuracy = tf.math.maximum(1. - (loss / (tf.reduce_mean(R*R))), 0.)

In [32]:
batched = True

In [33]:
if batched:
    n_steps = len(set(the_dataset.index.get_level_values(0)))* training_epochs
else:
    n_steps = training_epochs

In [39]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    
    if not TRAINING:
        saver.restore(sess, "/tmp/model.ckpt")
        print("model restored")
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    per_losses = []
    per_vars = []
    with coord.stop_on_exception():
        while not coord.should_stop():
            best_loss = np.inf
            best_step = -1

            inp, rets = next_element

            feed_dict = {
                X: inp,
                R: rets
            }
            if TRAINING:
                fetches = [train_op, loss, global_step, per_var] # accuracy
                _, loss_val, i, var_val = sess.run(fetches)
            else:
                fetches = [loss, global_step, per_var] # accuracy
                loss_val, i, var_val = sess.run(fetches)
            per_losses.append(loss_val)
            per_vars.append(var_val)
            #if loss_val < best_loss:
            #   best_loss = loss_val
            #    best_step = i


            if i % display_step == 0 and i > 0:
                if True:
                    the_mean_loss = np.mean(per_losses)
                    the_mean_var = np.mean(per_vars)
                    mean_r2 = 1. - (the_mean_loss / the_mean_var)
                else:
                    the_sum_loss = np.sum(per_losses)
                    the_sum_var = np.sum(per_vars)
                    mean_r2 = 1. - (the_sum_loss / the_sum_var)
                    print("Different Methodology")
                per_losses = []
                per_vars = []
                
                if TRAINING:
                    save_path = saver.save(sess, "/tmp/model.ckpt")
                    phase='Training'
                else:
                    phase='Testing'
                
                print("%s Equal-Date-Weighted R^2: %.4f, Epoch %i/%i" % (phase, mean_r2, i // display_step, training_epochs))
                
            global_step = (global_step + 1)

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
model restored
Testing Equal-Date-Weighted R^2: 0.0572, Epoch 3/10
